In [1]:
import csv
import lzstring
from collections import namedtuple, Counter
import json
from memoize import memoize
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# [num unique urls, num unique urls typed, total visits, total typed, first visit time, last visit time]
domaininfo = namedtuple('domaininfo', ['num_unique_urls', 'num_unique_urls_typed', 'total_visits', 'total_typed', 'first_visit_time', 'last_visit_time'])

decompressFromEncodedURIComponent = lzstring.LZString().decompressFromEncodedURIComponent

filepath = 'difficultyselectionexp_may31_11am.csv'
reader = csv.DictReader(open(filepath))

def extract_domain_visit_info(domain_visit_info_compressed):
  domain_visit_info = json.loads(decompressFromEncodedURIComponent(domain_visit_info_compressed))
  output = {}
  for k,v in domain_visit_info.items():
    linedata = domaininfo(*v)
    output[k] = linedata
  return output

alldata = []

for alldata_item in reader:
  if alldata_item['selected_difficulty'] not in ['nothing', 'easy', 'medium', 'hard']:
    continue
  if alldata_item['domain_visit_info_compressed'] == None or len(alldata_item['domain_visit_info_compressed']) == 0:
    continue
  alldata_item['domain_visit_info'] = extract_domain_visit_info(alldata_item['domain_visit_info_compressed'])
  alldata.append(alldata_item)


In [3]:
training_data = alldata[:round(len(alldata)*0.8)]
test_data = alldata[round(len(alldata)*0.8):]
print(len(training_data))
print(len(test_data))

98
25


In [4]:
def extract_labels_alldata(data):
  return np.array([line['selected_difficulty'] for line in data])

@memoize
def get_most_common_label():
  label_to_count = Counter()
  for line in training_data:
    label = line['selected_difficulty']
    label_to_count[label] += 1
  sorted_by_count = sorted(label_to_count.items(), key=lambda x: x[1], reverse=True)
  return sorted_by_count[0][0]

@memoize
def get_most_visited_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += info.total_visits
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

@memoize
def get_most_common_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += 1
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

def get_num_visits_for_domain(domain_visit_info, domain):
  info = domain_visit_info.get(domain, None)
  if info != None:
    return info.total_visits
  return 0

def extract_features_for_user(domain_visit_info):
  domains = get_most_common_domains()
  visits_for_domains = np.array([get_num_visits_for_domain(domain_visit_info, x) for x in domains])
  visits_for_domains = np.divide(visits_for_domains, np.sum(visits_for_domains))
  return visits_for_domains

def extract_features_alldata(data):
  output = []
  for line in data:
    domain_visit_info = line['domain_visit_info']
    features = extract_features_for_user(domain_visit_info)
    output.append(features)
  return np.array(output)


In [5]:
def get_percent_correct(predicted_labels, actual_labels):
  if len(predicted_labels) != len(actual_labels):
    raise 'need predicted and actual labels to have same lengths'
  total = len(actual_labels)
  correct = 0
  for p,a in zip(predicted_labels, actual_labels):
    if p == a:
      correct += 1
  return correct / total

def test_baseline_classifier():
  most_common_label = get_most_common_label()
  predictions = [most_common_label for line in test_data]
  actual = extract_labels_alldata(test_data)
  percent_correct = get_percent_correct(predictions, actual)
  print('baseline classifier accuracy:', percent_correct)

def test_classifier(clf, str=None):
  actual = extract_labels_alldata(test_data)
  features_test = extract_features_alldata(test_data)
  predictions = clf.predict(features_test)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier testing accuracy:', percent_correct)

def training_error_classifier(clf, str=None):
  actual = extract_labels_alldata(training_data)
  features_train = extract_features_alldata(training_data)
  predictions = clf.predict(features_train)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier training accuracy:', round(percent_correct, 2))

def to_int_categorical(dt):
  # {'easy', 'hard', 'medium', 'nothing'}
  cat_dt = []
  for item in dt:
    if item == 'nothing':
      cat_dt.append(0)
    elif item == 'easy':
      cat_dt.append(1)
    elif item == 'medium':
      cat_dt.append(2)
    else:
      cat_dt.append(3)
  return np.array(cat_dt)
    
test_baseline_classifier()

baseline classifier accuracy: 0.48


In [6]:
labels_train = extract_labels_alldata(training_data)
features_train = extract_features_alldata(training_data)

clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(features_train, labels_train)
test_classifier(clf, 'RF')

clf = KNeighborsClassifier(n_neighbors=3, p=1)
clf.fit(features_train, labels_train)
test_classifier(clf, 'KNN')

RF classifier testing accuracy: 0.28
KNN classifier testing accuracy: 0.56


In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import to_categorical

labels_test = to_categorical(to_int_categorical(extract_labels_alldata(test_data)), num_classes=4)
features_test = extract_features_alldata(test_data)

labels_train = to_categorical(to_int_categorical(extract_labels_alldata(training_data)), num_classes=4)
features_train = extract_features_alldata(training_data)

model = Sequential()

model.add(Dense(100, activation='relu', input_dim=100))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(features_train, labels_train,
          epochs=100,
          batch_size=32)
score = model.evaluate(features_test, labels_test, batch_size=32)

predictions = model.predict(features_test, batch_size=32)

print(predictions)
print(labels_test)
print(score)

Epoch 1/100
98/98 [==============================] - 0s 5ms/step - loss: 1.3985 - acc: 0.2245
Epoch 2/100
98/98 [==============================] - 0s 55us/step - loss: 1.3865 - acc: 0.2653
Epoch 3/100
98/98 [==============================] - 0s 69us/step - loss: 1.3676 - acc: 0.2959
Epoch 4/100
98/98 [==============================] - 0s 61us/step - loss: 1.3452 - acc: 0.3980
Epoch 5/100
98/98 [==============================] - 0s 85us/step - loss: 1.3555 - acc: 0.3878
Epoch 6/100
98/98 [==============================] - 0s 64us/step - loss: 1.3245 - acc: 0.4184
Epoch 7/100
98/98 [==============================] - 0s 82us/step - loss: 1.3143 - acc: 0.4082
Epoch 8/100
98/98 [==============================] - 0s 74us/step - loss: 1.2957 - acc: 0.4184
Epoch 9/100
98/98 [==============================] - 0s 80us/step - loss: 1.3050 - acc: 0.3980
Epoch 10/100
98/98 [==============================] - 0s 75us/step - loss: 1.2938 - acc: 0.4286
Epoch 11/100
98/98 [==============================

98/98 [==============================] - 0s 93us/step - loss: 1.2903 - acc: 0.4184
Epoch 86/100
98/98 [==============================] - 0s 110us/step - loss: 1.2841 - acc: 0.4286
Epoch 87/100
98/98 [==============================] - 0s 76us/step - loss: 1.2974 - acc: 0.4184
Epoch 88/100
98/98 [==============================] - 0s 73us/step - loss: 1.2811 - acc: 0.4184
Epoch 89/100
98/98 [==============================] - 0s 83us/step - loss: 1.2856 - acc: 0.4184
Epoch 90/100
98/98 [==============================] - 0s 84us/step - loss: 1.2887 - acc: 0.4082
Epoch 91/100
98/98 [==============================] - 0s 104us/step - loss: 1.2841 - acc: 0.4184
Epoch 92/100
98/98 [==============================] - 0s 94us/step - loss: 1.2904 - acc: 0.4184
Epoch 93/100
98/98 [==============================] - 0s 111us/step - loss: 1.3071 - acc: 0.4184
Epoch 94/100
98/98 [==============================] - 0s 90us/step - loss: 1.2621 - acc: 0.4184
Epoch 95/100
98/98 [==============================